In [1]:
import os

from utils import calc_3d_pmapper, scale_descriptors

from sklearn.model_selection import train_test_split

from miqsar.estimators.wrappers import MIWrapperMLPRegressor, miWrapperMLPRegressor
from miqsar.estimators.attention_nets import AttentionNetRegressor
from miqsar.estimators.mi_nets import MINetRegressor, miNetRegressor

from sklearn.metrics import r2_score

Read data

In [2]:
dataset_file = 'datasets/CHEMBL1075104.smi'

os.mkdir('descriptors')

path = os.path.join('descriptors', 'tmp')
os.mkdir(path)

Set hyperparameters

In [3]:
n_epoch = 500
batch_size = 128
lr = 0.001
weight_decay = 0.01
seed = 42
init_cuda = True
ncpu = 5
random_state = 42

Calculate descriptors for single conformation and train SI model

In [4]:
bags, labels, molid = calc_3d_pmapper(dataset_file, nconfs=1, stereo=False, path=path, ncpu=ncpu)
x_train, x_test, y_train, y_test, molid_train, molid_test = train_test_split(bags, labels, molid, random_state=random_state)
x_train, x_test = scale_descriptors(x_train, x_test)

ndim = (x_train[0].shape[-1], 256, 128, 64)

att_net = MINetRegressor(ndim=ndim, init_cuda=init_cuda)
att_net.fit(x_train, y_train, n_epoch=n_epoch, dropout=0.9, batch_size=batch_size, weight_decay=weight_decay, lr=lr)

predictions = att_net.predict(x_test)

print('3D/SI/Net: r2_score = {:.2f}'.format(r2_score(y_test, predictions)))

/home/zankov/anaconda3/envs/main/lib/python3.6/site-packages/torch_optimizer/yogi.py:117: UserWarning: This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629431274/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  grad = grad.add(group['weight_decay'], p.data)


3D/SI/Net: r2_score = -28.20


Calculate descriptors for multiple conformations and train MI model

In [5]:
bags, labels, molid = calc_3d_pmapper(dataset_file, nconfs=5, stereo=False, path=path, ncpu=ncpu)
x_train, x_test, y_train, y_test, molid_train, molid_test = train_test_split(bags, labels, molid, random_state=random_state)
x_train, x_test = scale_descriptors(x_train, x_test)

# train 3D/MI/Bag-AttentionNet
ndim = (x_train[0].shape[-1], 256, 128, 64)
det_ndim = (64,)

att_net = AttentionNetRegressor(ndim=ndim, det_ndim=det_ndim, init_cuda=init_cuda)
att_net.fit(x_train, y_train, n_epoch=n_epoch, dropout=0.8, batch_size=batch_size, weight_decay=weight_decay, lr=lr)

predictions = att_net.predict(x_test)
instance_weights = att_net.get_instance_weights(x_test)

print('3D/MI/Bag-AttentionNet: r2_score = {:.2f}'.format(r2_score(y_test, predictions)))

3D/MI/Bag-AttentionNet: r2_score = -51.95
